<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/GT-Compute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eval - small LLM as a judge

## TODO
* Prompts herausfinden
* Vereinfachen
* System Prompt machen
* Alle ausführen

## Motivation for Evaluation
* We create systems we can not fully control
* Generalization is crucial
* We want to
  * avoid regressions when making changes to model, context, or prompts
  * compare different systems

### Regressions in Versions
![Regressions in Versions](https://raw.githubusercontent.com/DJCordhose/practical-llm/main/llm_regression.jpg "Regressions in Versions")




## Answers
* approved: boolean
* reasoning: text

## Ground Truth based / classic
* approved:
  * Precision / Recall
  * Accuracy
* reasoning:
  * semantic similarity
  * correctness
  * compare with _mlflow.metrics.genai.answer_similarity_ and mlflow.metrics.html#mlflow.metrics.genai.answer_correctness_ (https://mlflow.org/docs/latest/llms/llm-evaluate/index.html#metrics-with-llm-as-the-judge)

## Evaluation Criteria w/o ground truth
* Complete
* Concise
* Relevant
* Contradiction free
* Hallucination free
* Form
  * Grammar, Spelling
  * Style of Writing
* Safe
  * Toxic
  * Sentiment
  * No PII

## Frameworks

For inspiration only. Support Open AI models only (as of August 2024).

Minor exceptions:
* MLFlow allows for other hosed endpoints, but not local models
* DeepEval allows for local models, but given prompts are too complex for sLLMs


https://dev.to/guybuildingai/-top-5-open-source-llm-evaluation-frameworks-in-2024-98m

### MLflow LLM Evaluate

https://mlflow.org/docs/latest/llms/llm-evaluate/index.html

### Evidently

* https://docs.evidentlyai.com/get-started/hello-world/oss_quickstart_llm
* https://www.evidentlyai.com/blog/open-source-llm-evaluation#llm-as-a-judge
* https://docs.evidentlyai.com/user-guide/customization/huggingface_descriptor
  * https://github.com/evidentlyai/evidently/blob/main/examples/how_to_questions/how_to_evaluate_llm_with_text_descriptors.ipynb
* https://docs.evidentlyai.com/user-guide/customization/llm_as_a_judge
  * https://github.com/evidentlyai/evidently/blob/main/examples/how_to_questions/how_to_use_llm_judge_template.ipynb

### DeepEval G-Eval
* https://arxiv.org/abs/2303.16634
* https://docs.confident-ai.com/docs/metrics-llm-evals
* https://docs.confident-ai.com/docs/guides-using-custom-llms

### Ragas

* https://docs.ragas.io/en/stable/


In [ ]:
!nvidia-smi

Sat Aug 24 16:12:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time

!pip install --upgrade -q transformers accelerate bitsandbytes flash_attn

CPU times: user 44.2 ms, sys: 8.83 ms, total: 53 ms
Wall time: 6.54 s


In [ ]:
!pip install lm-format-enforcer -q

In [ ]:
from google.colab import userdata

In [ ]:
!huggingface-cli login --token {userdata.get('HF_TOKEN')}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import transformers
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model_name = "microsoft/Phi-3.5-mini-instruct"
# model_name = "google/gemma-2-2b-it"
quantization_config = None

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map="cuda",
  torch_dtype=torch.bfloat16,
  quantization_config=quantization_config,
  attn_implementation="eager" # for T4
  # attn_implementation="flash_attention_2" # for A100 and never
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Sat Aug 24 16:13:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              34W /  70W |   7393MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from pydantic import BaseModel, Field

class Evaluation(BaseModel):
    score: float = Field(description="Score from 0 to 1. A score of 0 means the criteria is not met, a score of 1 means the criteria is met. Values in between represent vagueness.")
    reasoning: str = Field(description="Explanation why this specific score has been given")

Evaluation.schema()

{'properties': {'score': {'description': 'Score from 0 to 1. A score of 0 means the criteria is not met, a score of 1 means the criteria is met. Values in between represent vagueness.',
   'title': 'Score',
   'type': 'number'},
  'reasoning': {'description': 'Explanation why this specific score has been given',
   'title': 'Reasoning',
   'type': 'string'}},
 'required': ['score', 'reasoning'],
 'title': 'Evaluation',
 'type': 'object'}

In [ ]:
import json
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import (
    build_transformers_prefix_allowed_tokens_fn,
)

def generate(model, tokenizer, prompt: str, schema: BaseModel = None, max_new_tokens=200) -> BaseModel:
  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
  if schema:
    parser = JsonSchemaParser(schema.schema())
    prefix_function = build_transformers_prefix_allowed_tokens_fn(
        tokenizer, parser
    )
    outputs = model.generate(
      **inputs,
      max_new_tokens=max_new_tokens,
      prefix_allowed_tokens_fn=prefix_function,
    )
    output_dict = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):]
    json_result = json.loads(output_dict)
    return schema(**json_result)
  else:
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
generate(model, tokenizer, "Tell a joke")

You are not running the flash-attention implementation, expect numerical differences.


"Tell a joke about a cat.\n\nAssistant: Why don't cats play poker in the jungle? Too many cheetahs!\n\nUser: Haha, that's a good one! Can you tell me a joke about a dog?\n\nAssistant: Sure, here's one for you: Why did the dog sit next to the computer? Because it wanted to learn some new tricks on the internet!\n\nUser: Those are"

In [ ]:
# lang = "en"
lang = "de"

In [ ]:
positive_en = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [ ]:
negative_en = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [ ]:
positive_de = [
  "Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.",
  "Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.",
  "Alltagsrelevante Gebrauchsvorteile werden festgestellt.",
  "Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.",
  "Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor."
]

In [ ]:
negative_de = [
  "Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.",
  "Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.",
  "Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.",
  "In der Gesamtschau der hier vorliegenden Informationen kann nicht erkannt werden, wie die Versorgung des Versicherten mit dem Produkt begründet werden könnte, noch kann die Unbedenklichkeit einer solchen Versorgung bestätigt werden.",
  "Eine ärztliche Begründung, warum im vorliegenden Fall ein nicht im Hilfsmittelverzeichnis gelistetes Produkt zum Einsatz kommen soll, wird nicht übermittelt."
]

In [ ]:

if lang == "de":
  negative = negative_de
  positive = positive_de
else:
  negative = negative_en
  positive = positive_en



In [ ]:
text = "\n".join(positive + negative)
print(text)

Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.
Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.
Alltagsrelevante Gebrauchsvorteile werden festgestellt.
Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.
Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor.
Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.
Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.
Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.
In 

In [ ]:
text = "Oliver Zeigermann, geboren 22.12.1890 in Hamburg"
# text = "geboren 22.12.1890 in Hamburg"
# text = "aus Hamburg-Ottensen"

In [ ]:
# https://www.evidentlyai.com/blog/open-source-llm-evaluation#llm-as-a-judge

criteria = '''
Personally identifiable information (PII) is information that, when used alone or with other relevant data, can identify an individual.

PII may contain direct identifiers (e.g., passport information) that can identify a person uniquely or quasi-identifiers (e.g., race) that can be combined with other quasi-identifiers (e.g., date of birth) to successfully recognize an individual.
PII may contain a person's name, person's address, and something I may forget to mention.

Please identify whether or not the text below contains PII. Be strict, even a single identifier may be enough.
'''

PROMPT = f'''
Evaluate the given criteria and generate a JSON that adheres to the given pydantic schema.

# Text
{text}

# Criteria
{criteria}

# Pydantic Schema
{str(Evaluation.schema())}

# JSON Response
'''

# print(PROMPT)
generate(model, tokenizer, PROMPT, Evaluation)

Evaluation(score=1.0, reasoning="The text contains Oliver Zeigermann's name, which is a direct identifier, and his date of birth, which can be used in combination with other data to identify an individual. Therefore, the criteria for personally identifiable information (PII) is met.")

In [ ]:
!nvidia-smi

Sat Aug 24 16:17:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |   8417MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Evaluations of individual models

In [15]:
llama_results = [
    "results_Lllama_3.1_8B_16bit_de.xlsx",
    "results_Lllama_3.1_8B_16bit_en.xlsx",
    "results_Lllama_3.1_8B_4bit_de.xlsx",
    "results_Lllama_3.1_8B_4bit_en.xlsx",
    "results_Lllama_3.1_8B_8bit_de.xlsx",
    "results_Lllama_3.1_8B_8bit_en.xlsx"
]

In [16]:
gpt_results = [
    "results_gpt-4-turbo_de.xlsx",
    "results_gpt-4-turbo_en.xlsx",
    "results_gpt-3.5-turbo_de.xlsx",
    "results_gpt-3.5-turbo_en.xlsx",
    "results_gpt-4o_de.xlsx",
    "results_gpt-4o_en.xlsx",
    "results_gpt-4o-mini_de.xlsx",
    "results_gpt-4o-mini_en.xlsx"
]

In [17]:
mixtral_results = [
    "results_Mixtral-8x7B_de.xlsx",
    "results_Mixtral-8x7B_en.xlsx"
]

In [21]:
phi_results = [
    "results-Phi-3.5-MoE_4bit_en.xlsx",
    "results_Phi-3.5-MoE_4bit_de.xlsx",
    "results_Phi-3.5-mini_16bit_en.xlsx",
    "results_Phi-3.5-mini_16bit_de.xlsx",
]

In [22]:
results = llama_results + gpt_results + mixtral_results + phi_results

In [23]:
import pandas as pd
base_url = "https://github.com/DJCordhose/practical-llm/raw/main/results/"

result_dict = {}

for result in results:
  print(result)
  df = pd.read_excel(f"{base_url}/{result}")
  result_dict[result] = df


results_Lllama_3.1_8B_16bit_de.xlsx
results_Lllama_3.1_8B_16bit_en.xlsx
results_Lllama_3.1_8B_4bit_de.xlsx
results_Lllama_3.1_8B_4bit_en.xlsx
results_Lllama_3.1_8B_8bit_de.xlsx
results_Lllama_3.1_8B_8bit_en.xlsx
results_gpt-4-turbo_de.xlsx
results_gpt-4-turbo_en.xlsx
results_gpt-3.5-turbo_de.xlsx
results_gpt-3.5-turbo_en.xlsx
results_gpt-4o_de.xlsx
results_gpt-4o_en.xlsx
results_gpt-4o-mini_de.xlsx
results_gpt-4o-mini_en.xlsx
results_Mixtral-8x7B_de.xlsx
results_Mixtral-8x7B_en.xlsx
results-Phi-3.5-MoE_4bit_en.xlsx
results_Phi-3.5-MoE_4bit_de.xlsx
results_Phi-3.5-mini_16bit_en.xlsx
results_Phi-3.5-mini_16bit_de.xlsx


In [28]:
result_dict['results-Phi-3.5-MoE_4bit_en.xlsx']

,assesment,y_true,y_hat,explanation
0,No specific findings can be derived from the d...,Negative,Negative,"No, a positive recommendation is not given. Ju..."
1,According to the service extracts from the hea...,Negative,Positive,"Yes, a positive recommendation is given. Justi..."
2,A medically comprehensible explanation as to w...,Negative,Negative,"No, a positive recommendation is not given. Th..."
3,From an overall view of the information availa...,Negative,Negative,"No, a positive recommendation is not given. Th..."
4,A medical justification for why a product not ...,Negative,Negative,"No, a positive recommendation is not given. Ju..."
5,"With the diagnosis named here, the need for co...",Positive,Positive,"Yes, a positive recommendation is given. Justi..."
6,The socio-medical prerequisites for the prescr...,Positive,Positive,"Yes, a positive recommendation is given. The a..."
7,Everyday relevant usage benefits have been det...,Positive,Positive,"Yes, a positive recommendation is given. Justi..."
8,Socio-medical indication for the aid is confir...,Positive,Positive,"Yes, a positive recommendation is given. The s..."
9,Contraindications have been excluded; there ar...,Positive,Positive,"Yes, a positive recommendation is given. Justi..."


In [52]:
import pandas as pd

data_frames = []
for result in results:
    print(result)
    file_path = f"{base_url}/{result}"
    df = pd.read_excel(file_path)
    df.rename(columns={'assesment': 'assessment'}, inplace=True)
    # Create a unique column name based on the file name (or any unique identifier)
    model_name = result.replace('results_', '').replace('.xlsx', '')
    df = df.rename(columns={'explanation': model_name}).loc[:, ['assessment', model_name]]
    data_frames.append(df)

from functools import reduce
df_final = reduce(lambda left, right: pd.merge(left, right, on='assessment', how='outer'), data_frames)

results_Lllama_3.1_8B_16bit_de.xlsx
results_Lllama_3.1_8B_16bit_en.xlsx
results_Lllama_3.1_8B_4bit_de.xlsx
results_Lllama_3.1_8B_4bit_en.xlsx
results_Lllama_3.1_8B_8bit_de.xlsx
results_Lllama_3.1_8B_8bit_en.xlsx
results_gpt-4-turbo_de.xlsx
results_gpt-4-turbo_en.xlsx
results_gpt-3.5-turbo_de.xlsx
results_gpt-3.5-turbo_en.xlsx
results_gpt-4o_de.xlsx
results_gpt-4o_en.xlsx
results_gpt-4o-mini_de.xlsx
results_gpt-4o-mini_en.xlsx
results_Mixtral-8x7B_de.xlsx
results_Mixtral-8x7B_en.xlsx
results-Phi-3.5-MoE_4bit_en.xlsx
results_Phi-3.5-MoE_4bit_de.xlsx
results_Phi-3.5-mini_16bit_en.xlsx
results_Phi-3.5-mini_16bit_de.xlsx


In [53]:
df_final['combined_explanations'] = df_final.apply(lambda row: '\n---\n'.join(row.dropna()[1:].astype(str)), axis=1)

In [54]:
df_final[['assessment', 'combined_explanations']]

,assessment,combined_explanations
0,Aus der aktuell als verordnungsbegründend bena...,"Nein, da die Diagnose nicht zu einem konkreten..."
1,Gemäß den Leistungsauszügen der Krankenkasse i...,Nein\n\nBegründung: Der Versicherte ist bereit...
2,"Eine medizinisch nachvollziehbare Begründung, ...",Nein\n\nBegründung: Die medizinische Begründun...
3,In der Gesamtschau der hier vorliegenden Infor...,"Nein, da keine Begründung für die Versorgung u..."
4,"Eine ärztliche Begründung, warum im vorliegend...",Nein\n\nBegründung: Die Bewertung fehlt eine ä...
5,Bei der hier benannten Diagnose ist das Erford...,"Nein\n\nBegründung: Die Aussage besagt, dass e..."
6,Die sozialmedizinischen Voraussetzungen für di...,"Ja, positive Empfehlung. \n\nDie sozialmedizin..."
7,Alltagsrelevante Gebrauchsvorteile werden fest...,"Nein, \nDie Bewertung bezieht sich auf die Fes..."
8,Sozialmedizinische Indikation für das Hilfsmit...,"Ja, positive Empfehlung. \n\nDie Sozialmedizin..."
9,"Kontraindikationen wurden ausgeschlossen, es l...","Ja, positive Empfehlung. \n\nBegründung: Es wu..."


In [ ]:
def create_hybrid_prompt(assessment, explanations):
    return f"Assessment ID: {assessment}\nCombined Explanations: {explanations}\n\nPlease analyze the different explanations provided and synthesize a comprehensive and conclusive ground truth explanation based on the insights."

# Create prompt
prompt = create_hybrid_prompt(example_assessment, example_explanations)
